#### Source used: https://medium.com/@crcrpar/optuna-fastai-tabular-model-001-55777031e288

# Set Environment

## Install Packages

In [1]:
!pip3 install numpy
!pip3 install pandas
!pip3 install torch
!pip3 install optuna
!pip3 install scikit-learn==0.24.0
# !pip3 install -Uqq fastbook
!pip3 install fastai==1.0
!pip3 install --user graphviz
print("All Packages Installed Successfully")

All Packages Installed Successfully


## Import Packages

# Load the Dataset

In [ ]:
raw_data = pd.read_csv("data.csv")
raw_data.head()

# Optuna Optimization

## Define Variables

In [1]:
!pip3 install fastai==1.0.55
!pip3 install optuna

In [4]:
"""Optimize fastai tabular learner for Rossman without Pruning Callback."""
import datetime
import optuna
from functools import partial

from fastai.tabular import *
from fastai.callbacks import TrackerCallback

allow_pruning = False

class FastAIPruningCallback(TrackerCallback):
    def __init__(self, learn, trial, monitor):
        # type: (Learner, optuna.trial.Trial, str) -> None

        super(FastAIPruningCallback, self).__init__(learn, monitor)

        self.trial = trial

    def on_epoch_end(self, epoch, **kwargs):
        # type: (int, Any) -> None

        value = self.get_monitor_value()
        if value is None:
            return

        self.trial.report(value, step=epoch)
        if self.trial.should_prune():
            message = 'Trial was pruned at epoch {}.'.format(epoch)
            raise optuna.structs.TrialPruned(message)

path = "./"

raw_data = pd.read_csv('data.csv')
raw_data['y'].replace(1, 1, inplace=True)
raw_data['y'].replace(2, 2, inplace=True)
raw_data['y'].replace(3, 2, inplace=True)
raw_data['y'].replace(4, 2, inplace=True)
raw_data['y'].replace(5, 2, inplace=True)
# Percentage to split by for training
perc = 80

# Set for training the model
train_df = raw_data.head(int(len(raw_data)*(perc/100)))

# Set for testing the model later to get the real accuracy
test_df = raw_data.tail(int(len(raw_data)*((100-perc)/100)))

n = len(train_df)

procs= [FillMissing, Categorify, Normalize]

cont_vars = list(raw_data.columns.values)
cont_vars.remove("Unnamed: 0")
cont_vars.remove('y')

dep_var = 'y'
df = train_df[cont_vars + [dep_var, 'Unnamed: 0']].copy()
cut = train_df['Unnamed: 0'][(train_df['Unnamed: 0'] == train_df['Unnamed: 0'][len(test_df)])].index.max()
valid_idx = range(cut)

data = (TabularList.from_df(df, path=path, cont_names=cont_vars, procs=procs,)
                .split_by_idx(valid_idx)
                .label_from_df(cols=dep_var, label_cls=FloatList, log=True)
                .add_test(TabularList.from_df(test_df, path=path, cont_names=cont_vars))
                .databunch())

max_log_y = np.log(np.max(train_df['y']) * 1.2)
y_range = torch.tensor([0, max_log_y]).to(torch.device('cuda:0'))


def objective(trial):
    # Objective is `exp_rmspe`.
    n_layers = trial.suggest_int('n_layers', 1, 5)
    layers = []
    ps = []
    for i in range(n_layers - 1):
        # Number of units
        n_units = trial.suggest_categorical(
            'n_units_layer_{}'.format(i), [5, 20, 50, 56, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1100, 1200])
        layers.append(n_units)
        # Dropout ratio
        p = trial.suggest_discrete_uniform(
            'dropout_p_layer_{}'.format(i), 0, 1, 0.05)
        ps.append(p)

    emb_drop = trial.suggest_discrete_uniform('emb_drop', 0, 1, 0.05,)
    callback_fns = []
    if allow_pruning:
        callback_fns.append(
            partial(FastAIPruningCallback, trial=trial, monitor='exp_rmspe')
        )
    learn = tabular_learner(
        data, layers=layers, ps=ps, emb_drop=emb_drop, y_range=y_range,
        metrics=exp_rmspe, callback_fns=callback_fns, silent=True)

    learn.fit_one_cycle(5, 1e-3, wd=0.2)

    return learn.validate()[-1].item()

study = optuna.create_study(
        study_name='fastai-optuna-no-pruning',
        direction='minimize')

study.optimize(objective, n_trials=100)

if allow_pruning:
        pruned_trials = [
            t for t in study.trials if t.state == optuna.structs.TrialState.PRUNED
        ]
        complete_trials = [
            t for t in study.trials if t.state == optuna.structs.TrialState.COMPLETE
        ]
        print('Study statistics: ')
        print('  Number of finished trials: ', len(study.trials))
        print('  Number of pruned trials: ', len(pruned_trials))
        print('  Number of complete trials: ', len(complete_trials))

print('Best trial:')
trial = study.best_trial

print('  Value: ', trial.value)

print('  Params: ')
for key, value in trial.params.items():
    print('    {}: {}'.format(key, value))

print('  User attrs:')
for key, value in trial.user_attrs.items():
    print('    {}: {}'.format(key, value))

best_params = study.best_params
best_params['exp_rmspe'] = study.best_value

df = study.trials_dataframe()
df.to_csv('trials.csv')

[I 2021-02-14 15:40:42,056] A new study created in memory with name: fastai-optuna-no-pruning


[I 2021-02-14 15:40:49,219] Trial 0 finished with value: 0.2583947777748108 and parameters: {'n_layers': 4, 'n_units_layer_0': 5, 'dropout_p_layer_0': 0.65, 'n_units_layer_1': 1100, 'dropout_p_layer_1': 0.55, 'n_units_layer_2': 200, 'dropout_p_layer_2': 0.1, 'emb_drop': 1.0}. Best is trial 0 with value: 0.2583947777748108.


[I 2021-02-14 15:40:55,795] Trial 1 finished with value: 0.1423892080783844 and parameters: {'n_layers': 3, 'n_units_layer_0': 800, 'dropout_p_layer_0': 0.45, 'n_units_layer_1': 800, 'dropout_p_layer_1': 0.7000000000000001, 'emb_drop': 0.9}. Best is trial 1 with value: 0.1423892080783844.


[I 2021-02-14 15:41:01,123] Trial 2 finished with value: 0.3737221956253052 and parameters: {'n_layers': 1, 'emb_drop': 0.30000000000000004}. Best is trial 1 with value: 0.1423892080783844.


[I 2021-02-14 15:41:08,741] Trial 3 finished with value: 0.2840338349342346 and parameters: {'n_layers': 5, 'n_units_layer_0': 600, 'dropout_p_layer_0': 0.8500000000000001, 'n_units_layer_1': 200, 'dropout_p_layer_1': 0.5, 'n_units_layer_2': 300, 'dropout_p_layer_2': 0.8500000000000001, 'n_units_layer_3': 56, 'dropout_p_layer_3': 0.05, 'emb_drop': 0.1}. Best is trial 1 with value: 0.1423892080783844.


...

[I 2021-02-14 15:51:27,872] Trial 96 finished with value: 0.16024968028068542 and parameters: {'n_layers': 4, 'n_units_layer_0': 1000, 'dropout_p_layer_0': 0.4, 'n_units_layer_1': 900, 'dropout_p_layer_1': 0.05, 'n_units_layer_2': 1200, 'dropout_p_layer_2': 0.15000000000000002, 'emb_drop': 0.30000000000000004}. Best is trial 42 with value: 0.11471424996852875.


[I 2021-02-14 15:51:34,722] Trial 97 finished with value: 0.13685567677021027 and parameters: {'n_layers': 3, 'n_units_layer_0': 800, 'dropout_p_layer_0': 0.30000000000000004, 'n_units_layer_1': 5, 'dropout_p_layer_1': 0.45, 'emb_drop': 0.25}. Best is trial 42 with value: 0.11471424996852875.


[I 2021-02-14 15:51:41,502] Trial 98 finished with value: 0.1534617841243744 and parameters: {'n_layers': 3, 'n_units_layer_0': 400, 'dropout_p_layer_0': 0.35000000000000003, 'n_units_layer_1': 300, 'dropout_p_layer_1': 0.0, 'emb_drop': 0.45}. Best is trial 42 with value: 0.11471424996852875.


[I 2021-02-14 15:51:48,349] Trial 99 finished with value: 0.20992082357406616 and parameters: {'n_layers': 3, 'n_units_layer_0': 5, 'dropout_p_layer_0': 0.45, 'n_units_layer_1': 500, 'dropout_p_layer_1': 0.15000000000000002, 'emb_drop': 0.5}. Best is trial 42 with value: 0.11471424996852875.


Best trial:
  Value:  0.11471424996852875
  Params: 
    n_layers: 3
    n_units_layer_0: 700
    dropout_p_layer_0: 0.2
    n_units_layer_1: 5
    dropout_p_layer_1: 0.05
    emb_drop: 0.25
  User attrs:
